# Assigment 1: Primitive Sensitivity Analysis
#### Moritz Kick, Rami El Immam, Maximilian Nowak

## Implementation of the model
### 1. Implement SEIR model

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import ipywidgets as widgets
from IPython.display import display

beta = 14/9 # infection rate
alpha = 7/3 # incubation rate
gamma = 7/9 # recovery rate
i0 = 1000 # initial number of infected individuals

# Define initial conditions
N = 80e6
S0 = N - i0
E0 = 0
I0 = i0
R0 = 0
y0 = [S0, E0, I0, R0]

# Define the system of differential equations
def seir_model(y, t, beta, alpha, gamma, N):
    S, E, I, R = y
    dSdt = -beta * S * I / N
    dEdt = beta * S * I / N - alpha * E
    dIdt = alpha * E - gamma * I
    dRdt = gamma * I
    return [dSdt, dEdt, dIdt, dRdt]

# Define time points (100 Weeks)
t = np.linspace(0, 100, 100)


### 2. & 3. Plot and play with the parameters

In [2]:
# Define sliders for parameters
beta_slider = widgets.FloatSlider(
    value=0.5, min=0.0, max=5.0, step=0.01, description='beta:')
alpha_slider = widgets.FloatSlider(
    value=0.2, min=0.0, max=5.0, step=0.01, description='alpha:')
gamma_slider = widgets.FloatSlider(
    value=0.1, min=0.0, max=5.0, step=0.01, description='gamma:')
i0_slider = widgets.IntSlider(
    value=1000, min=0, max=N, step=1e6, description='i0:')

# Define a function to update the plot


def update_plot(beta, alpha, gamma, i0):
    sol = odeint(seir_model, y0, t, args=(beta, alpha, gamma, N))
    S, E, I, R = sol.T
    plt.figure(figsize=(10, 6))
    plt.plot(t, S, 'b', label='Susceptible')
    plt.plot(t, E, 'r', label='Exposed')
    plt.plot(t, I, 'g', label='Infected')
    plt.plot(t, R, 'm', label='Recovered')
    plt.title('SEIR Model')
    plt.xlabel('Time (weeks)')
    plt.ylabel('Population')
    plt.legend()
    plt.show()


# Display the plot and sliders
interactive_plot = widgets.interactive(
    update_plot, beta=beta_slider, alpha=alpha_slider, gamma=gamma_slider, i0=i0_slider)
output = interactive_plot.children[-1]
output.layout.height = '350px'
display(interactive_plot)


interactive(children=(FloatSlider(value=0.5, description='beta:', max=5.0, step=0.01), FloatSlider(value=0.2, …

## Perturbation of a nominal value
### 4. Define Hk in mathematical terms